# **Combination VSM and BM25 Search Engine**
_Group 69: Alain Ameer, Alexander Coudijzer, Bradley Aldous and Florin Pintilii._

# Contents
1. Introduction
1. Data Loading
1. Indexing Method
  - Pre-Processing
  - Term-Frequency Matrix
1. Retrieval Method
  - Query Vectorization
  - Vector Space Model (VSM)
  - Best Match 25 (BM25)
1. Command Line Implementation
1. Bibliography

# Introduction

This search engine enables users to enter a query as a string, and select whether VSM or BM25 is used as the retrieval model to return search results. The dataset used is the SciFact dataset, available from https://ir-datasets.com/beir.html#beir/scifact (MacAvaney et al, 2021).

All relevant libraries are imported and datasets installed in the code cell below. The nltk library has been sourced from Bird and colleagues (2009).

In [1]:
%%capture
!pip install ir_datasets
!pip install unidecode

import numpy as np
import ir_datasets
import string
import re
import unidecode
import pandas as pd
from numpy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer
from operator import itemgetter

import nltk             # nltk has been sourced from (Bird et al, 2009).
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

# Data Loading

The characteristics of the SciFact dataset are displayed by the code cell below. There are 5183 documents, 809 associated queries, and 919 query relevance judgements ("qrels").

In [2]:
dataset = ir_datasets.load("beir/scifact/train")

print('\n', dataset.docs_count(), 'documents with format:\n\t', 
      dataset.docs_cls().__annotations__)
print('\n', dataset.queries_count(), 'queries with format:\n\t', 
      dataset.queries_cls().__annotations__)
print('\n', dataset.qrels_count(), 'query relevance judgements with format:\n\t', 
      dataset.qrels_cls().__annotations__)


 5183 documents with format:
	 OrderedDict([('doc_id', <class 'str'>), ('text', <class 'str'>), ('title', <class 'str'>)])

 809 queries with format:
	 OrderedDict([('query_id', <class 'str'>), ('text', <class 'str'>)])

 919 query relevance judgements with format:
	 OrderedDict([('query_id', <class 'str'>), ('doc_id', <class 'str'>), ('relevance', <class 'int'>), ('iteration', <class 'str'>)])


The code cell below loads the data.

In [3]:
%%capture
for doc in dataset.docs_iter()[:1]: None
for q in dataset.queries_iter(): None
for qrel in dataset.qrels_iter(): None

The text of the documents have been collected into a list variable `documents` in the code cell below. The text alone has been taken for simplicity at this stage, however the `doc_id` will later be required for evaluation.

In [4]:
corpus = []
for doc in dataset.docs_iter():
    corpus.append([doc[0],doc[1],doc[2]])
documents = [doc[1] for doc in corpus]

print('\nNumber of documents: ' + str(len(documents))
      + '\n\nExample documents: "' + documents[0][:50] + '..."'
      + '\n\t\t   "' + documents[1][:50] + '..."'
      + '\n\t\t   "' + documents[2][:50] + '..."')


Number of documents: 5183

Example documents: "Alterations of the architecture of cerebral white ..."
		   "Myelodysplastic syndromes (MDS) are age-dependent ..."
		   "ID elements are short interspersed elements (SINEs..."


The text of the queries have likewise been collected into a list variable `queries` in the code cell below.

In [5]:
queries = []
for q in dataset.queries_iter():
    queries.append(q[1])
    
print('\nNumber of queries: ' + str(len(queries))
      + '\n\nExample queries: "' + queries[0][:50] + '..."'
      + '\n\t\t "' + queries[1][:50] + '..."'
      + '\n\t\t "' + queries[2][:50] + '..."')


Number of queries: 809

Example queries: "0-dimensional biomaterials lack inductive properti..."
		 "1 in 5 million in UK have abnormal PrP positivity...."
		 "1-1% of colorectal cancer patients are diagnosed w..."


# Indexing Method

### _Indexing Method: Pre-Processing_

The first stage of the indexing method is to pre-process text.

The code cell below defines the `pre_processing` function. The following stages are applied to the text of each document:

[A] Text is cleaned.
  1. Numbers are removed.
  1. Punctuation is removed and characters are converted to lowercase.
  1. Multiple whitespaces are replaced with single whitespaces. Leading and training whitespaces are also removed.

[B] Tokenization.

[C] Stop word removal.

[D] Lemmatization.

[E] Stemming.

[F] Accents are removed.

In [6]:
def pre_processing(docs):
    
    # [A] A function to clean text is defined.
    def clean_text(text):
        # [1] Numbers are removed.
        text_nonum = re.sub(r'\d+', '', text) 
        # [2] Punctuation is removed and characters converted to lowercase.
        text_nopunct = "".join([char.lower() for char in text_nonum
                                if char not in string.punctuation])
        # [3] Multiple whitespaces are replaced with single whitespaces.
        #     Leading and trailing whitespaces are removed.
        text_no_multispace = re.sub('\s+', ' ', text_nopunct).strip()
        return text_no_multispace
    
    # [A] Each document is passed through the "clean_text" function.
    documents_cleaned = []
    for d in docs:
        x = clean_text(d)
        documents_cleaned.append(x)
    
    # [B] Each document is tokenized.
    documents_tokenized = []
    for d in documents_cleaned:
        x = word_tokenize(d)
        documents_tokenized.append(x)
    
    # [C] Each document has stop words removed.
    documents_sw_removed = []
    stop_words = set(stopwords.words('english'))
    for a in documents_tokenized:
      filtered_sentence = [w for w in a if not w in stop_words]
      filtered_sentence = []
      for w in a:  
        if w not in stop_words:  
          filtered_sentence.append(w)
      documents_sw_removed.append(filtered_sentence)
    
    # [D] Each document undergoes lemmatization.
    documents_lemmatized = []
    wordnet_lemmatizer = WordNetLemmatizer()
    for d in documents_sw_removed:
      y = []
      for word in d:
        x = wordnet_lemmatizer.lemmatize(word)
        y.append(x)
      documents_lemmatized.append(y)
    
    # [E] Each document undergoes stemming.
    snow_stemmer = SnowballStemmer(language='english')
    documents_stemmed = []
    for d in documents_lemmatized:
      stems = []
      for word in d:
        x = snow_stemmer.stem(word)
        stems.append(x)
      documents_stemmed.append(stems)
    
    # [F] Accents are removed from each document.
    documents_decoded = []
    for d in documents_stemmed:
      a = []
      for word in d:
        x = unidecode.unidecode(word)
        a.append(x)
      documents_decoded.append(a)
    
    # [G] Processed documents are collected into "documents_pre_processed"
    documents_pre_processed = []
    for doc in documents_decoded:
      documents_pre_processed.append(' '.join(doc))
    return documents_pre_processed

The code cell below applies the previously defined `pre_processing` function to both the documents and the queries. Both documents and queries need to be pre-processed in the same way to ensure compatability.

In [7]:
documents_pre_processed = pre_processing(documents)
queries_pre_processed = pre_processing(queries)

The code cell below demonstrates the results of applying the `pre_processing` function.

In [8]:
print('\nDocument prior to pre-processing:\n' + '\t"' + documents[0][:97]
      + '..."\n\nDocument after pre-processing has been performed:\n'
      + '\t"' + documents_pre_processed[0][:64] + '..."\n\n'
      + 'Query prior to pre-processing:\n' + '\t"' + queries[0] + '"\n\n'
      + 'Query after pre-processing has been performed:\n'
      + '\t"' + queries_pre_processed[0] + '"')


Document prior to pre-processing:
	"Alterations of the architecture of cerebral white matter in the developing human brain can affect..."

Document after pre-processing has been performed:
	"alter architectur cerebr white matter develop human brain affect..."

Query prior to pre-processing:
	"0-dimensional biomaterials lack inductive properties."

Query after pre-processing has been performed:
	"dimension biomateri lack induct properti"


### _Indexing Method: Term-Frequency Matrix_

The pre-processed documents are vectorised in the code cell below to create a term-frequency (TF) matrix, `term_frequency_matrix` which is a pandas dataframe. Each row represents a document and each column represents a term in the corpus. The columns collectively represent the vocabulary of the entire corpus. Each cell represents the frequency with which the term of the column occurs in the document of the row.

In [9]:
vectorizer = CountVectorizer()
documents_vectorized = vectorizer.fit_transform(documents_pre_processed).toarray()
vocabulary = vectorizer.get_feature_names_out()

term_frequency_matrix = pd.DataFrame(documents_vectorized, columns = vocabulary)
term_frequency_matrix

,42,aa,aaa,aaaatpas,aaafamili,aab,aabenhus,aacr,aacrthi,aactinin,...,zygos,zygot,zymographi,zymosan,zymosaninduc,zyxin,zz,zzw,zzz,zzzw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Retrieval Method

### _Retrieval Method: Query Vectorization and Vocabularization_

The code cell below retrieves the query ID numbers. These IDs are combined with the respective query text that has previously been pre-processed, into the list variable `queries`.

In [10]:
qid = []
for q in dataset.queries_iter():
  qid.append(q[0])
qid = [eval(x) for x in qid]  # Reading in query IDs and converting to int
queries = [list(a) for a in zip(qid, queries_pre_processed)]

print('\nNumber of queries with ID: ' + str(len(queries))
      + '\n\nExample queries with ID: ' + str(queries[0])
      + '\n\t\t\t ' + str(queries[1])
      + '\n\t\t\t ' + str(queries[2]))


Number of queries with ID: 809

Example queries with ID: [0, 'dimension biomateri lack induct properti']
			 [2, 'million uk abnorm prp posit']
			 [4, 'colorect cancer patient diagnos region distant metastasi']


The code cell below likewise retrieves the document ID numbers. These IDs are combined with the respective document text that has previously been pre-processed, into the list variable `documents_id`.

In [11]:
documents_id = []
for doc in corpus:
    documents_id.append([doc[0],doc[1]])

print('\nNumber of documents with ID: ' + str(len(documents_id))
      + '\n\nExample documents with ID: ' + str(documents_id[0])[:50] + "...']"
      + '\n\t\t\t   ' + str(documents_id[1])[:50] + "...']"
      + '\n\t\t\t   ' + str(documents_id[2])[:50] + "...']")


Number of documents with ID: 5183

Example documents with ID: ['4983', 'Alterations of the architecture of cereb...']
			   ['5836', 'Myelodysplastic syndromes (MDS) are age-...']
			   ['7912', "ID elements are short interspersed eleme...']


The code cell below sets the query number to be tested.

In [12]:
query_num = 1

In [13]:
query_to_be_tested = queries[query_num][1]

The code cell below defines a function `query_vectorizer_vocabularizer` that obtains the vocabulary of the query to be tested `q_vocabulary`. The query is subsequently converted into a term vector `q_tf_vector` with the same number of elements as columns of the previously defined `term_frequency_matrix` (31075 terms). The elements are a $0$ when the respective term is absent and $1$ when it is present.

The `q_tf_vector` will subsequently be used as the query representation in the VSM model, and `q_vocabulary` will be used as the query representation in the BM25 model.

In [14]:
def query_vectorizer_vocabularizer(query_to_be_vectorized):

  q_vectorizer = CountVectorizer()
  q = q_vectorizer.fit_transform([query_to_be_vectorized])
  q_vocabulary = q_vectorizer.get_feature_names_out()
  
  print('\t   Vocabulary: ' + str(q_vocabulary) + '')

  q_tf_vector = np.array([])
  for t in term_frequency_matrix.columns:
      if t in q_vocabulary: q_tf_vector = np.append(q_tf_vector,1)
      else: q_tf_vector = np.append(q_tf_vector,0)

  print('Term-Frequency Vector: ' + str(q_tf_vector) + '' 
        + ' (Sum = ' + str(round(q_tf_vector.sum(),2)) 
        + ', Length = ' + str(len(q_tf_vector)) + ')')
  
  return q_tf_vector, q_vocabulary

The previously defined `query_vectorizer_vocabularizer` function is run below for the query to be tested, and returns the variables `q_vocabulary` and `q_tf_vector`.

In [15]:
print('   Query To Be Tested: "' + str(query_to_be_tested) + '"')
q_tf_vector, q_vocabulary = query_vectorizer_vocabularizer(query_to_be_tested)

   Query To Be Tested: "million uk abnorm prp posit"
	   Vocabulary: ['abnorm' 'million' 'posit' 'prp' 'uk']
Term-Frequency Vector: [0. 0. 0. ... 0. 0. 0.] (Sum = 5.0, Length = 31075)


### Retrieval Method: Vector Space Model (VSM)

The code cell below defines variables that will be used by the Vector Space Model (VSM) retrieval model.

- `dfs` represents document frequency. `dfs` is a pandas series where each row corresponds to a term in the vocabulary of the corpus (31075 terms), and each value represents the number of documents that contain that term.

- `N` is an integer representing the total number of documents in the corpus (5183).

- `idfs` is the inverse document frequency (IDF). It is also a pandas series of the same shape as `dfs`. It is calculated by taking logarithm to base 10 of the result of dividing the total number of documents `N` by the document frequency `dfs` ($log_{10}(N/dfs)$).

In [16]:
dfs = (term_frequency_matrix > 0).sum(axis = 0) # Document frequency.
N = term_frequency_matrix.shape[0] # Total number of documents.
idfs = np.log10(N/dfs) # Inverse document frequency.
print("\nidfs:\n\n" + str(idfs))


idfs:

42           3.714581
aa           2.600638
aaa          3.112521
aaaatpas     3.237460
aaafamili    3.714581
               ...   
zyxin        3.714581
zz           3.714581
zzw          3.714581
zzz          3.714581
zzzw         3.714581
Length: 31075, dtype: float64


All the elements of the previously defined `term_frequency_matrix` are multiplied by the inverse document frequency `idfs`, to obtain a term frequency–inverse document frequency matrix `tf_idf_matrix`.

In [17]:
tf_idf_matrix = np.array(term_frequency_matrix * idfs) #TF-IDF matrix.

print("\ntf_idf_matrix:\n\n" + str(tf_idf_matrix))
print(str(tf_idf_matrix.shape[0]) + " rows by " + str(tf_idf_matrix.shape[1]) + " columns.")
print("\nThe sum of all elements in tf_idf_matrix is " + str(int(tf_idf_matrix.sum().sum()))
      + ", \nwhich is up from " + str(term_frequency_matrix.sum().sum())
      + " for term_frequency_matrix.")


tf_idf_matrix:

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
5183 rows by 31075 columns.

The sum of all elements in tf_idf_matrix is 1044106, 
which is up from 643219 for term_frequency_matrix.


As with the `term_frequency_matrix`, the previously defined query term-frequency vector `q_tf_vector` likewise needs to be multiplied by the inverse document frequency, to obtain a TF-IDF vector `q_tf_idf_vector`. The sum of this vector is now 9.6 rather than 5 for the number of terms it contains. This operation is performed by the function defined in the code cell below.

The function defined in the code cell below proceeds to calculate the VSM scores for all the documents in the corpus. It does this by calculating the cosine similarity between the document vectors of TF-IDF weights, stored in the TF-IDF matrix `tf_idf_matrix`, and query vector of TF-IDF weights `q_tf_idf_vector` previously defined.

The cosine similarity is calculated by dividing the dot product of the two vectors by the product of their Euclidean normalisation.

In [18]:
def VSM(q_tf_vector): # Calculations inferred from Baeza-Yates and Ribeiro-Neto (2011).

  q_tf_idf_vector = (q_tf_vector * idfs).values
  print('Query TF-IDF Vector:   ' + str(q_tf_idf_vector) 
        + ' (Sum = ' + str(round(q_tf_idf_vector.sum(),2)) 
        + ', Length = ' + str(len(q_tf_idf_vector)) + ')')

  VSM_scores = []
  for d in tf_idf_matrix:
      VSM_scores.append(np.dot(d, q_tf_idf_vector)/(norm(d)*norm(q_tf_idf_vector)))

  return VSM_scores

The previously defined "VSM" function is run below and returns the variable `scores`. The query TF-IDF vector it initially creates is displayed.

In [19]:
scores = VSM(q_tf_vector)

scores_string = ''
for s in scores[100:103]:
  scores_string += str(s) + ', '
print('"scores" variable:     [...' + scores_string 
      + '...] (Sum = ' + str(round(sum(scores),2)) 
      + ', Length = ' + str(len(scores)) + ')')

Query TF-IDF Vector:   [0. 0. 0. ... 0. 0. 0.] (Sum = 9.61, Length = 31075)
"scores" variable:     [...0.0, 0.0, 0.0, ...] (Sum = 13.62, Length = 5183)


A function is defined in the code cell below to display a table. The table contains the document ID, document text and retrieval score for each document. The documents are sorted in descending retrieval scores, such that the most similar documents to the query are at the top of the table.

In [20]:
def display_ranking(top_n, scores, score_type):

  documents_scored = []

  for (d,n,s) in zip(documents_id, range(len(documents_id)), scores):
    documents_scored.append([])
    documents_scored[n].append(int(d[0]))
    documents_scored[n].append(d[1])
    documents_scored[n].append(s)

  documents_scored = sorted(documents_scored, # Sort by descending score.
                            key = itemgetter(2), reverse = True)

  documents_scored = pd.DataFrame(documents_scored, 
                                  columns = ['Doc_ID', 
                                             'Text', 
                                             score_type + " Score"]).set_index('Doc_ID')
  return documents_scored.head(top_n)

The previously defined `display_ranking` function is called in the code cell below, to display the top 10 documents.

In [21]:
display_ranking(10, scores, "VSM")

,Text,VSM Score
Doc_ID,,
13734012,OBJECTIVES To carry out a further survey of ar...,0.364472
26059876,"Ku70, a known nonhomologous end-joining (NHEJ)...",0.352551
13770184,"BACKGROUND The Global Burden of Diseases, Inju...",0.238112
3716075,BACKGROUND Dengue is the most common arbovirus...,0.168597
14610165,Antisense transcription is widespread in many ...,0.128051
23531592,By the year 2000 there will be six million pre...,0.109300
11880289,BACKGROUND Age-specific effects of mammographi...,0.108722
11936877,Soil erosion is a major environmental threat t...,0.095828
841371,OBJECTIVE To assess the robustness of patient ...,0.090755


### Retrieval Method: Best Match 25 (BM25)

The code cell below defines variables that will be used by the Best Match 25 (BM25) retrieval model.

- `dls` represents document length. `dls` is a list containing the respective document length for each document indexed in the previously defined `term_frequency_matrix`. The length has been calculated by summing over the columns in the `term_frequency_matrix`, which has the benefit of ignoring stop words that have been  previously removed by the `pre_processing` function.

- `avgdl` represents average document length, calculated as mean of `dls`.

- Standard values have been used for hyperparameters `k_1` and `b` of 1.2 and 0.8 respectively.

The numerator is calculated by multiplying the previously defined TF weights in `term_frequency_matrix` by the result of adding 1 to the hyperparameter `k_1`: $TF * (k_1 + 1)$.

The denominator is calculated by the following formula: $TF + (k_1 * (1 - b + (b * (dls/avgdl))))$.

The result of dividing the numerator by the denominator is multiplied by the inverse document frequencies previously defined as `idfs`.

The resulting BM25 weights are collected in the matrix `BM25_matrix`, displayed below.

In [22]:
k_1 = 1.2 # Standard hyperparameter.
b = 0.8   # Standard hyperparameter.
dls = term_frequency_matrix.sum(axis=1).tolist() # Document length.
avgdl = np.mean(dls) # Mean document length.

# Applying the BM25 formula:
numerator = np.array(term_frequency_matrix * (k_1 + 1))
denominator = np.array(term_frequency_matrix) \
              + np.array(k_1 * ((1 - b) + (b * (dls / avgdl)))).reshape(N,1)
BM25_matrix = (numerator / denominator) * np.array(idfs)
BM25_matrix = pd.DataFrame(BM25_matrix, columns = vocabulary)

In [23]:
print("\nBM25_matrix (first 5 rows and 5 columns):\n\n" + str(BM25_matrix.iloc[:5,:5]))
print(str(BM25_matrix.shape[0]) + " rows by " + str(BM25_matrix.shape[1]) + " columns.")
print("\nThe sum of all elements in BM25_matrix is " + str(int(BM25_matrix.sum().sum()))
      + ", \nwhich is up from " + str(term_frequency_matrix.sum().sum())
      + " for term_frequency_matrix.")


BM25_matrix (first 5 rows and 5 columns):

    42   aa  aaa  aaaatpas  aaafamili
0  0.0  0.0  0.0       0.0        0.0
1  0.0  0.0  0.0       0.0        0.0
2  0.0  0.0  0.0       0.0        0.0
3  0.0  0.0  0.0       0.0        0.0
4  0.0  0.0  0.0       0.0        0.0
5183 rows by 31075 columns.

The sum of all elements in BM25_matrix is 748189, 
which is up from 643219 for term_frequency_matrix.


The code cell below defines a function that calculates the BM25 scores for all the documents in the corpus. It does this by first filtering the query vocabulary `q_vocabulary` to only include terms present in the vocabulary of the corpus, to produce the new variable `q_vocabulary_in_docs`. BM25 scores are then calculated as the sum of all the BM25 weights in the previously defined BM25_matrix, for all terms that appear in the query.

In [24]:
def BM25(q_vocabulary): # Calculations inferred from Baeza-Yates and Ribeiro-Neto (2011).

  q_vocabulary_in_docs = [term for term in q_vocabulary   # Query terms not in 
                          if term in BM25_matrix.columns] # documents are removed.

  BM25_scores = BM25_matrix[q_vocabulary_in_docs].sum(axis = 1)

  return BM25_scores

The previously defined `BM25` function is run below and returns the variable `scores`, which has the same format as returned by the previous `VSM` function, a pandas series with 5183 elements.

In [25]:
scores = BM25(q_vocabulary)

A table is displayed by the code cell below, which contains the document ID, document text and BM25 score for each document. The documents are sorted in descending BM25 scores, such that the most similar documents to the query are at the top of the table.

In [26]:
display_ranking(10, scores, "BM25")

,Text,BM25 Score
Doc_ID,,
13734012,OBJECTIVES To carry out a further survey of ar...,13.009034
26059876,"Ku70, a known nonhomologous end-joining (NHEJ)...",7.475744
841371,OBJECTIVE To assess the robustness of patient ...,4.750421
14610165,Antisense transcription is widespread in many ...,4.739075
32481310,Hemolysates of erythrocytes from more than a q...,4.244058
4828631,BACKGROUND High body-mass index (BMI) predispo...,4.153526
18617259,We report a case of preclinical variant Creutz...,4.084513
13770184,"BACKGROUND The Global Burden of Diseases, Inju...",3.523895
11271123,Endometrial cancer is associated with numeric ...,3.474263


# Command Line Implementation

A function `search_engine` has been defined in the code cell below to enable the user to enter a string and receive a display of ranked documents. The function enables the user to select whether they want the VSM or BM25 retrieval model to be used, and to specify the number of top results they want displayed.

In [27]:
def search_engine(query_to_be_tested, retrieval_model = "VSM", top_n = 10):

  print('\n   Query to be tested: "' + str(query_to_be_tested) + '"')

  pre_processed_query = pre_processing([query_to_be_tested])[0]

  print('\n  Pre-Processed Query: "' + str(pre_processed_query) + '"')
  
  q_tf_vector, q_vocabulary = query_vectorizer_vocabularizer(pre_processed_query)

  if retrieval_model == "VSM":  scores = VSM(q_tf_vector)
  if retrieval_model == "BM25": scores = BM25(q_vocabulary)

  return display_ranking(top_n, scores, retrieval_model)

The `search_engine` function is called below for the query "Brown fat is a specialized fat depot" and VSM retrieval model.

In [28]:
search_engine("Brown fat is a specialized fat depot?", "VSM")


   Query to be tested: "Brown fat is a specialized fat depot?"

  Pre-Processed Query: "brown fat special fat depot"
	   Vocabulary: ['brown' 'depot' 'fat' 'special']
Term-Frequency Vector: [0. 0. 0. ... 0. 0. 0.] (Sum = 4.0, Length = 31075)
Query TF-IDF Vector:   [0. 0. 0. ... 0. 0. 0.] (Sum = 8.28, Length = 31075)


,Text,VSM Score
Doc_ID,,
14865329,Brown fat is a specialized fat depot that can ...,0.618403
29381091,"Brown adipocytes dissipate energy, whereas whi...",0.463040
20532591,White adipose tissue displays high plasticity....,0.344243
38023457,Severe quantitative and qualitative brown adip...,0.313916
36558211,OBJECTIVES To explore the different characteri...,0.297875
26902591,Cancer-associated cachexia (CAC) is a wasting ...,0.201231
19205437,Uncoupling protein 1 (UCP1) plays a central ro...,0.170187
25974070,The amount and type of dietary fat have long b...,0.163213
13277118,BACKGROUND Polar bears (Ursus maritimus) are a...,0.160586


The `search_engine` function is called below for the query "Alterations of the architecture of cerebral white matter" and BM25 retrieval model.

In [29]:
search_engine("Alterations of the architecture of cerebral white matter!", "BM25")


   Query to be tested: "Alterations of the architecture of cerebral white matter!"

  Pre-Processed Query: "alter architectur cerebr white matter"
	   Vocabulary: ['alter' 'architectur' 'cerebr' 'matter' 'white']
Term-Frequency Vector: [0. 0. 0. ... 0. 0. 0.] (Sum = 5.0, Length = 31075)


,Text,BM25 Score
Doc_ID,,
4983,Alterations of the architecture of cerebral wh...,14.074497
8227227,The locations of cerebral infarctions were stu...,8.580971
18104691,"AIM This review explores the molecular, neurol...",7.578883
1472815,OBJECTIVE The purpose of our study was to inve...,7.182748
37583120,OBJECTIVE Obesity and being overweight during ...,7.135950
1412089,BACKGROUND Traditional T2 weighted MR imaging ...,7.117689
22107641,OBJECTIVE The purpose of this study was to det...,6.851754
17930286,OBJECTIVE To evaluate the association of overa...,6.771738
8672737,BACKGROUND AND PURPOSE Migraineurs are at incr...,6.007959


# Bibliography

Baeza-Yates, R. and Ribeiro-Neto, B. (2011). Modern Information Retrieval: the concepts and technology behind search. 2nd Edition. Harlow, Pearson.

Bird, S., Edward, L. & Ewan, K. (2009). Natural Language Processing with Python. California, O`Reilly Media Inc.

MacAvaney, S., Yates, A., Feldman, S., Downey, D., Cohan, A., & Goharian, N. (2021). Simplified Data Wrangling with ir-datasets, SIGIR ’21, July 11–15, 2021.